In [107]:
import pandas as pd
import sqlite3 as sql
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import folium

In [111]:
locator = Nominatim(user_agent="myGeocoder")
database = "db.sqlite3"
connection = sql.connect(database)
query = '''SELECT * FROM myadmin_clientrecord'''
df = pd.read_sql_query(query,connection)
df['ADDRESS'] = df["city"].str.cat([df["postcode"], df["state"]], sep=',')

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
big_df = pd.DataFrame(df)

# Combine "column1" and "column2" into a single DataFrame
combined_df = big_df[['latitude', 'longitude']].copy()
combined_df = combined_df.dropna()

map1 = folium.Map(
    location=[4.2105, 101.9758],
    tiles='cartodbpositron',
    zoom_start=7,
)
combined_df.apply(lambda row:folium.Marker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1